<a href="https://colab.research.google.com/github/samindelhi/CNN-Exercises/blob/main/Yolo11_Transfer_learning_With_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yolo11 Transfer Learning With Correction


In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.5 MB/s eta 0:00:00


In [10]:
import torch
import torchvision.transforms as T

from ultralytics import YOLO
from ultralytics.data.dataset import ClassificationDataset
from ultralytics.models.yolo.classify import ClassificationTrainer, ClassificationValidator, ClassificationPredictor

class CustomizedDataset(ClassificationDataset):
  """ A Customized dataset class for image classification with enhanced data augmentation transforms"""

  def __init__(self, root:str, args, augment:bool=False, prefix:str=""):
    """Initialize a customised classificaiton dataset with enhanced data augmentation transforms."""
    super().__init__(root, args, augment, prefix)
    train_transforms = T.Compose(
        [
            T.Resize((args.imgsz, args.imgsz)),
            T.RandomHorizontalFlip(p=args.fliplr),
            T.RandomVerticalFlip(p=args.flipud),
            T.RandAugment(interpolation=T.InterpolationMode.BILINEAR),
            T.ColorJitter(
                brightness=args.hsv_v,
                contrast=args.hsv_v,
                saturation=args.hsv_s,
                hue=args.hsv_h
            ),
            T.ToTensor(),
            T.Normalize(mean=torch.tensor(0), std=torch.tensor(1)),
            T.RandomErasing(p=args.erasing, inplace=True)]
    )
    # Add your custom validation transforms here
    val_transforms = T.Compose(
        [
            T.Resize((args.imgsz, args.imgsz)),
            T.ToTensor(),
            T.Normalize(mean=torch.tensor(0), std=torch.tensor(1)),
        ]
    )
    self.torch_transforms = train_transforms if augment else val_transforms

In [11]:
class CustomizedTrainer(ClassificationTrainer):
    """A customized trainer class for YOLO classification models with enhanced dataset handling."""

    def build_dataset(self, img_path: str, mode: str = "train", batch=None):
        """Build a customized dataset for classification training and the validation during training."""
        return CustomizedDataset(root=img_path, args=self.args, augment=mode == "train", prefix=mode)


class CustomizedValidator(ClassificationValidator):
    """A customized validator class for YOLO classification models with enhanced dataset handling."""

    def build_dataset(self, img_path: str, mode: str = "train"):
        """Build a customized dataset for classification standalone validation."""
        return CustomizedDataset(root=img_path, args=self.args, augment=mode == "train", prefix=self.args.split)


model = YOLO("yolo11n-cls.pt")
model.train(data="imagenet1000", trainer=CustomizedTrainer, epochs=10, imgsz=224, batch=64)
model.val(data="imagenet1000", validator=CustomizedValidator, imgsz=224, batch=64)

Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=imagenet1000, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7df9c2dda060>
curves: []
curves_results: []
fitness: 0.5633732676506042
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.4341317415237427, 'metrics/accuracy_top5': 0.6926147937774658, 'fitness': 0.5633732676506042}
save_dir: PosixPath('/content/runs/classify/val')
speed: {'preprocess': 0.3591832524953206, 'inference': 0.5528026696608183, 'loss': 0.00017929041923580625, 'postprocess': 0.00031769161682542016}
task: 'classify'
top1: 0.4341317415237427
top5: 0.6926147937774658

In [14]:
import gdown

# Google Drive file ID
file_id = "1TCU1nqgIe1R_dW6LTkRxlufHlCCazyJl"
# Download destination filename
output = "myfile.zip"

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1TCU1nqgIe1R_dW6LTkRxlufHlCCazyJl
From (redirected): https://drive.google.com/uc?id=1TCU1nqgIe1R_dW6LTkRxlufHlCCazyJl&confirm=t&uuid=8971e30c-8ef4-4c9e-ad9e-3f6b01a10123
To: /content/myfile.zip
100%|██████████| 63.9M/63.9M [00:00<00:00, 149MB/s]


'myfile.zip'

In [15]:
import zipfile

with zipfile.ZipFile("myfile.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

In [16]:
# 📁 2. Define paths (update with your dataset path)
train_dir = "/content/dataset/train"
val_dir = "/content/dataset/valid"
img_size = 299
batch_size = 32


In [17]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-cls.pt")  # load an official model
# model = YOLO("yolo11n.pt")  # load a custom model
model.train(data="/content/dataset/train", trainer=CustomizedTrainer, epochs=10, imgsz=224, batch=64)





Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/train, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7df9761cb8c0>
curves: []
curves_results: []
fitness: 0.978515625
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.95703125, 'metrics/accuracy_top5': 1.0, 'fitness': 0.978515625}
save_dir: PosixPath('/content/runs/classify/train4')
speed: {'preprocess': 0.05925074609347547, 'inference': 0.3078453554676841, 'loss': 8.326171840877805e-05, 'postprocess': 0.00016608593789158022}
task: 'classify'
top1: 0.95703125
top5: 1.0

In [ ]:
# CLI Implementation.Start training from a pretrained *.pt model.
# !yolo classify train data=imagenet10 model=yolo11n-cls.pt epochs=5 imgsz=224


In [18]:
metrics = model.val(data="/content/dataset/valid", validator=CustomizedValidator, imgsz=224, batch=64)


Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
WARNING ⚠️ Dataset 'split=train' not found at /content/dataset/valid/train
Found 364 images in subdirectories. Attempting to split...
Splitting /content/dataset/valid (2 classes, 364 images) into 80% train, 20% val...
Split complete in /content/dataset/valid_split ✅
train: /content/dataset/valid_split/train... found 290 images in 2 classes ✅ 
val: /content/dataset/valid_split/val... found 74 images in 2 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 880.9±268.5 MB/s, size: 25.0 KB)
val: Scanning /content/dataset/valid_split/val... 74 images, 0 corrupt: 100% ━━━━━━━━━━━━ 74/74 5.1Kit/s 0.0s
val: New cache created: /content/dataset/valid_split/val.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 2/2 0.9it/s 2.2s
                   all      0.851          1
Speed: 1.1ms prep

In [20]:
metrics.top1, metrics.top5

(0.8513513803482056, 1.0)

In [22]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolo11n-cls.pt")  # load an official model
model = YOLO("/content/runs/classify/train4/weights/best.pt")  # load a custom model

# Predict with the model
results = model("/content/dataset/valid/daisy/14698531521_0c2f0c6539_jpg.rf.787d556676fe7d11fe3909353835a730.jpg")  # predict on an image


image 1/1 /content/dataset/valid/daisy/14698531521_0c2f0c6539_jpg.rf.787d556676fe7d11fe3909353835a730.jpg: 224x224 daisy 1.00, dandelion 0.00, 5.0ms
Speed: 4.4ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
